In [95]:
from pyspark.sql import SparkSession
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('vizualizações3').getOrCreate()

23/11/10 22:11:59 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/11/10 22:11:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/10 22:12:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df_parquet = spark.read.parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_cities_skylines/total_video_publicado_semana/total_video_publicado_semana.parquet')
df_parquet.createOrReplaceTempView('TOTAL_PUBLICACOES')

In [5]:
spark.sql('SELECT * FROM TOTAL_PUBLICACOES WHERE NM_CANAL LIKE "C%"').show(truncate=False)

+----------------+----------------------------+------------+---------------+------------------------+
|SEMANA_TRADUZIDA|NM_CANAL                    |TOTAL_VIDEOS|DATA_PUBLICACAO|ID_CANAL                |
+----------------+----------------------------+------------+---------------+------------------------+
|Segunda-feira   |CANAL HAND - GAME WITH MUSIC|1           |2023-10-16     |UC7v51W2NoLFawizgcNuVBOA|
|Segunda-feira   |CANAL HAND - GAME WITH MUSIC|1           |2023-10-23     |UC7v51W2NoLFawizgcNuVBOA|
|Quarta-feira    |CANAL HAND - GAME WITH MUSIC|1           |2023-10-18     |UC7v51W2NoLFawizgcNuVBOA|
|Terça-feira     |CANAL HAND - GAME WITH MUSIC|1           |2023-10-17     |UC7v51W2NoLFawizgcNuVBOA|
|Sexta-feira     |CANAL HAND - GAME WITH MUSIC|1           |2023-10-20     |UC7v51W2NoLFawizgcNuVBOA|
|Sábado          |CANAL HAND - GAME WITH MUSIC|1           |2023-10-21     |UC7v51W2NoLFawizgcNuVBOA|
|Domingo         |CANAL HAND - GAME WITH MUSIC|1           |2023-10-22     |UC7v51

In [6]:
spark.sql('SELECT DATA_PUBLICACAO, sum(TOTAL_VIDEOS) FROM TOTAL_PUBLICACOES GROUP BY  DATA_PUBLICACAO').show()

+---------------+-----------------+
|DATA_PUBLICACAO|sum(TOTAL_VIDEOS)|
+---------------+-----------------+
|     2023-10-25|               23|
|     2023-10-16|               16|
|     2023-10-15|               10|
|     2023-10-18|               14|
|     2023-10-23|               24|
|     2023-10-21|               21|
|     2023-10-20|               13|
|     2023-10-19|               17|
|     2023-10-22|               15|
|     2023-10-17|               16|
|     2023-10-26|               22|
|     2023-10-24|               23|
|     2023-10-27|               21|
+---------------+-----------------+



In [7]:
spark.sql('SELECT COUNT(*) FROM TOTAL_PUBLICACOES where DATA_PUBLICACAO = "2023-10-25"  ' ).show(26)

+--------+
|count(1)|
+--------+
|      23|
+--------+



- Relação Comentários X Visualização (proporção) e Relação Visualizações X Likes (proporção)

In [36]:
assunto = 'assunto_cities_skylines'

In [37]:
df_visualizacoes_semana = spark.read.parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/{assunto}/total_visualizacoes_por_semana')
df_visualizacoes_semana.createOrReplaceTempView('METRICAS_VIDEO')

In [38]:
spark.sql(' SELECT *'
          ' FROM METRICAS_VIDEO ').show(truncate=False)

+---------------+-----------+---------------------------------------------------------------------------------------------+---------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+----------+-------------------+-----------------+-----------+-----------------------+---------------------+---------------+-------------+------------------------+
|NM_CANAL       |ID_VIDEO   |TITULO_VIDEO                                                                                 |TOTAL_CARACTERE_VIDEO|TAGS                

In [39]:
spark.sql(' SELECT ID_CANAL, NM_CANAL, count(*)'
          ' FROM METRICAS_VIDEO '
          ' GROUP BY ID_CANAL, NM_CANAL ORDER BY 3 DESC').show(truncate=False)

+------------------------+----------------------------+--------+
|ID_CANAL                |NM_CANAL                    |count(1)|
+------------------------+----------------------------+--------+
|UCrOH1V-FyMunBIMrKL0y0xQ|ChratosGameplay             |114     |
|UC7v51W2NoLFawizgcNuVBOA|CANAL HAND - GAME WITH MUSIC|83      |
|UCkXA2EoaXRFW18vrAE01BpQ|Betospa Gaming              |67      |
|UC1mk6EtfMjxR4eEZ7C43zTQ|Irmãos Cities               |67      |
|UCfY4ggUDPeecGSCNePyyVSA|MetalBear                   |58      |
|UCcn60fs5EMW8A_k9R-e4gEg|Cabrunco Games              |51      |
|UCOYJm7BsLV8VCkW2DC1MI5w|Lucas gameplays brasil      |51      |
|UCayiH2Y5JBEOSubqoQhufDA|Lives do Chratos            |48      |
|UCDGo_s0I-2JpO3o3xlkMQVA|Simularies                  |38      |
|UCrUcdH_bCfX77xpXHyq2fJw|Canal do Void               |37      |
|UCmg3_taw1BrGuo1ITlQe2hA|O Construtor                |36      |
|UCTrNnXivZU_1-aJqn1uNANg|DILSÃO                      |35      |
|UCe9jrI0YQ5SM6h5QRZ9FZlA

In [74]:
df_result = spark.sql(' SELECT ID_VIDEO, TOTAL_CARACTERE_VIDEO, TITULO_VIDEO,NM_CANAL, TOTAL_COMENTARIOS, TOTAL_LIKES , TOTAL_VISUALIZACOES '
          ' FROM METRICAS_VIDEO '
          ' WHERE data_extracao = "2023-10-27" ' 
          ' AND ID_CANAL in ("UCrOH1V-FyMunBIMrKL0y0xQ") ').toPandas()
fig = px.scatter(df_result, x='TOTAL_CARACTERE_VIDEO', y='TOTAL_VISUALIZACOES')
df_result.sort_values(by='TOTAL_VISUALIZACOES', ascending=False).head()


,ID_VIDEO,TOTAL_CARACTERE_VIDEO,TITULO_VIDEO,NM_CANAL,TOTAL_COMENTARIOS,TOTAL_LIKES,TOTAL_VISUALIZACOES
13,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,418,13988,120812
10,totHMIv6_hI,70,PEQUENA FORTUNA EM SERVIÇOS PÚBLICOS BÁSICOS 🏙...,ChratosGameplay,188,8978,71357
12,vy-upTpchig,63,PRIMEIRA PRAÇA DA CIDADE SAI DO PAPEL 🏙️ - Cit...,ChratosGameplay,201,7321,60786
2,9SGBZ7GE5EA,70,SUPER SUBÚRBIO FAZ ALCANÇAR 10000 HABITANTES 🏙...,ChratosGameplay,213,6855,54375
3,KfzZNkdyeEs,81,POLUIÇÃO FORÇA MIGRAÇÃO DA INDÚSTRIA PARA NOVO...,ChratosGameplay,124,6448,50789


In [75]:
fig.show()

- Comparação Likes X Visualizações e Cometários X Visualizações

In [76]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['Likes X Visualizações', 'Cometários X Visualizações', 'TOTAL_CARACTERE_VIDEO X VISUALIZAÇÔES'])
sc1 = px.scatter(df_result, x='TOTAL_LIKES', y='TOTAL_VISUALIZACOES',)
fig.add_trace(sc1['data'][0], row=1, col=1)
fig.update_xaxes(title_text='Likes', row=1, col=1)
fig.update_yaxes(title_text='Visualizações', row=1, col=1)

sc2 = px.scatter(df_result, x='TOTAL_COMENTARIOS', y='TOTAL_VISUALIZACOES')
fig.add_trace(sc2['data'][0], row=1, col=2)
fig.update_xaxes(title_text='Comentários', row=1, col=2)
fig.update_yaxes(title_text='Visualizações', row=1, col=2)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)

sc3 = px.scatter(df_result, x='TOTAL_CARACTERE_VIDEO', y='TOTAL_VISUALIZACOES')
fig.add_trace(sc3['data'][0], row=1, col=3)
fig.update_xaxes(title_text='TOTAL_CARACTERE_VIDEO', row=1, col=3)
fig.update_yaxes(title_text='Visualizações', row=1, col=3)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)



fig.show()

# Dia em que os vídeos são Públicados com alta frequência

In [78]:
df_dias_publicado = spark.read.parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_cities_skylines/total_video_publicado_semana')
df_dias_publicado.createOrReplaceTempView('PUBLICACAO_VIDEO')

In [85]:
spark.sql('select * from PUBLICACAO_VIDEO').show()

+----------------+--------------------+------------+---------------+--------------------+
|SEMANA_TRADUZIDA|            NM_CANAL|TOTAL_VIDEOS|DATA_PUBLICACAO|            ID_CANAL|
+----------------+--------------------+------------+---------------+--------------------+
|    Quarta-feira|BL4D3N | Jogos de...|           1|     2023-10-25|UCIv5D2ro1owj6P7O...|
|    Quinta-feira|Bad Influence gui...|           1|     2023-10-26|UC8FS3_qf6F194QTY...|
|     Terça-feira|KyoKat PC Gamepla...|           1|     2023-10-24|UCLAUYEY8ITQC8b47...|
|   Segunda-feira|CANAL HAND - GAME...|           1|     2023-10-16|UC7v51W2NoLFawizg...|
|   Segunda-feira|CANAL HAND - GAME...|           1|     2023-10-23|UC7v51W2NoLFawizg...|
|    Quarta-feira|CANAL HAND - GAME...|           1|     2023-10-18|UC7v51W2NoLFawizg...|
|     Terça-feira|CANAL HAND - GAME...|           1|     2023-10-17|UC7v51W2NoLFawizg...|
|     Sexta-feira|CANAL HAND - GAME...|           1|     2023-10-20|UC7v51W2NoLFawizg...|
|         

In [115]:
def indice_semana(dia: str) -> int: 
    dias_semana = {
        'Domingo': 1,
        'Segunda-feira' : 2,
        'Terça-feira' : 3,
        'Quarta-feira' : 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado' : 7
    }

    return dias_semana.get(dia)


In [101]:
indice_semana('Sábado')

7

In [119]:
df_publicacao_video  = spark.sql(' SELECT SEMANA_TRADUZIDA,  sum(TOTAL_VIDEOS) '
          ' FROM PUBLICACAO_VIDEO'
          ' GROUP BY SEMANA_TRADUZIDA ').toPandas()
df_publicacao_video['INDICE_SEMANA'] = df_publicacao_video['SEMANA_TRADUZIDA'].apply(indice_semana)
df_publicacao_video.sort_values(by='INDICE_SEMANA', inplace=True)
df_publicacao_video.head(7)


,SEMANA_TRADUZIDA,sum(TOTAL_VIDEOS),INDICE_SEMANA
5,Domingo,25,1
0,Segunda-feira,40,2
2,Terça-feira,39,3
1,Quarta-feira,37,4
6,Quinta-feira,39,5
4,Sexta-feira,34,6
3,Sábado,21,7


In [120]:
fig = px.bar(df_publicacao_video, x='SEMANA_TRADUZIDA',y='sum(TOTAL_VIDEOS)', text_auto=True)
fig.show()

# Total de Tags X Vísualizações 

# Tamanho dos títulos do Vídeos X Vísualizações

# Total Víews por Vídeo

# A duração do vídeo é importante para visualizações e interação (curtidas/comentários)?

# Desepenho dos vídeos (like, comentários, vísualizações) (Faça comparação)

# Desepenho dos Canais (like, comentários, vísualizações) Faça Comparação


- Métricas especificas TRENDS